In [33]:
import pandas as pd
import os
import json
import numpy as np
import re

In [34]:
## reading the Survey File
#surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\Test.csv",sep="," , skiprows=1)
surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\AmenityClubSurvey FINAL v2 Season Cleaned_deletedFirstTwoColumn.csv",sep="," , skiprows=1)
surveyData.head()

## Reading the Config File and Extracting the Information from there
config_file = open("C:\Projects\MSS\Loading _AmenityData_from_Survey\Python_Codes\ClubCorp\DiningRoom\diningColMapping.json")
dinningRoom_config = json.load(config_file)

In [35]:
def addToDayTimeMap (phonebook, key, value):
    if key in phonebook:
        x =  phonebook[key]
        phonebook[key] = x + "," + value
        
    else :
        phonebook[key] = value

In [36]:
def prepareHoursOfOperationJson(hoursOfOperationInfo, surveyData, isFirstDiningRoom, columnIndexIncreaseCountFor2ndDining):
    
    allHoursOfOperation = []
    
    for i in range(0, len(surveyData)) : 
        hoursOfOperationForEachRow = []
        hoursOfOperation = {}
        hoursOfOperationValue = {}

        season = "All Season"
        ## adding season value
        if season and not pd.isnull(season):
            seasonValue = {}
            seasonValue["value"] = season
            hoursOfOperationValue["Season"] = [seasonValue]
        
        
        ## creating day time map so that days with same open and closed time can handled properly
        dayTimeMap = {}    
        for j in range(0, len(hoursOfOperationInfo)):
            
            dayInfo = hoursOfOperationInfo[j]
            day = dayInfo["Day"]
            openTimeIndex = dayInfo["OpenTime"]
            closeTimeIndex = dayInfo["CloseTime"]
            
            if not isFirstDiningRoom :
                openTimeIndex = openTimeIndex+  columnIndexIncreaseCountFor2ndDining;
                closeTimeIndex = closeTimeIndex+  columnIndexIncreaseCountFor2ndDining;
                
            startTimeRaw = surveyData.iat[i, openTimeIndex]
            endTimeRaw = surveyData.iat[i, closeTimeIndex]
            
            startTime = ""
            endTime = ""
            if pd.notnull(startTimeRaw):
                startTime = str(startTimeRaw).strip()
            
            if pd.notnull(endTimeRaw):
                endTime = str(endTimeRaw).strip()
                
            
            
            if startTime == "Closed" or endTime == "Closed" :
                continue
            else :  
                addToDayTimeMap(dayTimeMap, startTime + "," + endTime, day)
            
        
        dayAndTimingArray = []
        
        for startAndEndTime, days in dayTimeMap.items():
            
            startTime,endTime = startAndEndTime.split(",")
            openDays = days.split(",")
            ## Adding Day and Timing
            dayAndTiming = {}
            dayAndTimingValue = {}
            for day in openDays :
                dayValue = {}
                dayValue["value"] = True
                dayAndTimingValue[day] = [dayValue]

            timing = {}
            timingValue = {}

            if startTime and not pd.isnull(startTime): 
                startTimeDict ={}
                startTimeDict["value"] = startTime
                timingValue["StartTime"] =  [startTimeDict]

            if endTime and not pd.isnull(endTime):
                endTimeDict ={}
                endTimeDict["value"] = endTime
                timingValue["EndTime"] =  [endTimeDict]
            
            ## Adding day part as "All Day"
            dayPart ={}
            dayPart["value"] = "All Day"
            timingValue["DayPart"] =  [dayPart]


            if timingValue :
                timing["value"] = timingValue
                dayAndTimingValue["Timing"] = [timing]

            if dayAndTimingValue : 
                dayAndTiming["value"] = dayAndTimingValue    
                dayAndTimingArray.append(dayAndTiming)
        
        hoursOfOperationValue["DayAndTiming"] = dayAndTimingArray
        if hoursOfOperationValue :
            hoursOfOperation["value"] = hoursOfOperationValue
            
        hoursOfOperationForEachRow.append(hoursOfOperation)
        allHoursOfOperation.append(json.dumps(hoursOfOperationForEachRow))
    
    return allHoursOfOperation

In [37]:
def determineUpscale (dataFrame):
    val = ""
    if dataFrame["Upscale/Casual"] == "Upscale":
        val = True
    return val

In [38]:
def determineCasual (dataFrame):
    val = ""
    if dataFrame["Upscale/Casual"] == "Casual":
        val = True
    return val

In [39]:
def determineBooleanValueForBar(dataFrame):
    val = ""
    if pd.notnull(dataFrame["Bar"]) :
        if dataFrame["Bar"] == "Yes":
            val = True
        elif dataFrame["Bar"].strip() == "":
            val = ""
        else:
            val = False
    return val

In [40]:
## Getting the output File name
output_fileName = dinningRoom_config['outputFileName']

## column index to be increased for getting other Golf Course information
columnIndexIncreaseCountFor2ndDining = dinningRoom_config['columnIndexIncreaseCountFor2ndDining']


common_column_indexes = []
column_names = []

## column index for 1st and 2nd 
columnIndexForFirstDiningRoom = []
columnIndexForSecondSecondDiningRoom = []

## getting common column information and golf course related column information from config file
common_column_count = len(dinningRoom_config['commonColumnInformation'])
dining_column_count = len(dinningRoom_config['diningRoomRelatedColumn'])

## Getting column index from config json file
for i in range(0, common_column_count):
    common_column_indexes.append(dinningRoom_config['commonColumnInformation'][i]['columnIndex'])
    column_names.append(dinningRoom_config['commonColumnInformation'][i]['outputColumnName'])
    
for i in range(0, dining_column_count):
    columnIndexForFirstDiningRoom.append(dinningRoom_config['diningRoomRelatedColumn'][i]['columnIndex'])
    columnIndexForSecondSecondDiningRoom.append(dinningRoom_config['diningRoomRelatedColumn'][i]['columnIndex'] + columnIndexIncreaseCountFor2ndDining)
    
    column_names.append(dinningRoom_config['diningRoomRelatedColumn'][i]['outputColumnName'])
    
## getting Hours of Related column information from config file
hoursOfOperationInfo = dinningRoom_config['HoursOfOperationInformation']


## Data for 1st Golf Course
diningRoom1 = surveyData.iloc[:,common_column_indexes + columnIndexForFirstDiningRoom]
diningRoom1.columns = column_names

diningRoom1['Index']=1

hrsOfOperation1 = prepareHoursOfOperationJson(hoursOfOperationInfo, surveyData, True, columnIndexIncreaseCountFor2ndDining)
diningRoom1['HrsOfOpeartion'] = hrsOfOperation1


## Data for 2nd Golf Course
diningRoom2 = surveyData.iloc[:, common_column_indexes + columnIndexForSecondSecondDiningRoom]
diningRoom2.columns = column_names

diningRoom2['Index']=2

hrsOfOperation2 = prepareHoursOfOperationJson(hoursOfOperationInfo, surveyData, False, columnIndexIncreaseCountFor2ndDining)
diningRoom2['HrsOfOpeartion'] = hrsOfOperation2

## Adding these 3 information together
allGolfSurveyData = pd.concat([diningRoom1, diningRoom2])
allGolfSurveyData['Status'] = 'Active'
allGolfSurveyData['Type'] = 'Dining'


## deleting all the rows for which name is blank or, Na or, 0
dataExceptBlankName = allGolfSurveyData[(allGolfSurveyData['Name'].str.lower() != 'na') & (allGolfSurveyData['Name'].str.lower() != 'n/a') & (allGolfSurveyData['Name'].notnull()) & (allGolfSurveyData['Name'] != '0')]

dataExceptBlankName['Casual'] = dataExceptBlankName.apply(determineCasual, axis=1)
dataExceptBlankName['Upscale'] = dataExceptBlankName.apply(determineUpscale, axis=1)
dataExceptBlankName['Bar_Boolean'] = dataExceptBlankName.apply(determineBooleanValueForBar, axis=1)

dataExceptBlankName.to_csv("C:/Projects/MSS/Loading _AmenityData_from_Survey/Python_Codes/ClubCorp/DiningRoom/" + output_fileName)

C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o